## Yoana D. - CSCI-E7 - 2017- Graduate project 
- The script will take a list of 12 accesion numbers for homologous proteins, 
    - obtain the fasta format text file of all the sequences, 
    - run sequence alignment by the MAFFT alignment software. 
    - output color coded alignment/visualization with ESPript
    - obtain any available pdb IDs for 3D structures from the Protein Data Bank. 
- The final output file will represent color-coded alignment of homologous protein sequences and a text file with:
    - pdb-ID numbers of available 3D structures for these proteins or homologous proteins
    - percent_identity
    - title of publications 
    

## Input / output data:
- input 1: list of accession numbers
- output file 1: sequences.fasta      (protein sequences in fasta format)
- output file 2: pdb_info.txt         (pdb information for each protein)
- output file 3: seq_alignment.fasta  (sequence alignment in fasta format)
- input 2: espript.inp                (input script to run ESPript software)
- output file 4: alignment.pdf        (colored version of sequence alignemnt)

In [1]:
#accession numbers for CENP-A homologous proteins
#obtained from blast search: https://blast.ncbi.nlm.nih.gov/Blast.cgi

cenp_a_an = ['NP_001800','XP_004029016.1','XP_003308962.1','XP_003270664 ',\
             'XP_001087306', 'XP_011239001.1', 'EHB18906', 'KFO23359.1',\
             'NP_012875', 'KTB18012.1', 'XP_452757', 'NP_983029']

## Steps 1: parse the list of numbers and write sequences into a fasta file
- input 1: list of protein accession numbers
- output file 1: sequences.fasta

In [2]:
from Bio import Entrez
from Bio import SeqIO

#provide your email as a string
Entrez.email = "dimitrova@crystal.harvard.edu"

#Step 1: save protein sequences in fasta file

for anumber in cenp_a_an:
    
    #specify from what db, what id, what type, what to return
    handle = Entrez.efetch(db="protein", id = anumber, retmod='text', rettype='fasta')
    
    #Create a record 
    record = SeqIO.read(handle, "fasta")
    handle.close()
    protein_seq = ('>'+ record.description + '\n' + record.seq + '\n')
    
    with open("sequences.fasta", "a") as out:      
        out.write(str(protein_seq))

## Step 2: search the Protein Data Bank for pdb files 
- input list of protein accession numbers
- output file2: pdb_info.txt

In [6]:
from prody import blastPDB

#use the same for loop as above to fetch the sequences for the PDB blast

for anumber in cenp_a_an:
    handle = Entrez.efetch(db="protein", id = anumber, retmod='text', rettype='fasta')
    record = SeqIO.read(handle, "fasta")
    handle.close()
    
#Step 2: use each sequence to search the pdb for structures
    blast_record = blastPDB(str(record.seq))
    
    # Best matches from Blast and other pdb hits
    best = blast_record.getBest()
    hits = blast_record.getHits()
    
    #write the output in a text file
    with open("pdb_info.txt", "a") as out: 
        out.write('\n' + "title: {0}, pdb_id: {1}, percent_identity: {2}"\
                  .format(best['title']+'\n',best['pdb_id']+'\n', best['percent_identity']))
        out.write('\n' + "all_hits: {}".format((list(hits))) + '')
      
        

@> Blast searching NCBI PDB database for "MGPRR..."
@> Blast search completed in 3.5s.
@> Blast searching NCBI PDB database for "MGPRR..."
@> Blast search completed in 11.1s.
@> Blast searching NCBI PDB database for "MGPRR..."
@> Blast search completed in 6.8s.
@> Blast searching NCBI PDB database for "MGPRR..."
@> Blast search completed in 3.6s.
@> Blast searching NCBI PDB database for "MGPRR..."
@> Blast search completed in 5.5s.
@> Blast searching NCBI PDB database for "MGPRR..."
@> Blast search completed in 8.7s.
@> Blast searching NCBI PDB database for "MDVHS..."
@> Blast search completed in 6.1s.
@> Blast searching NCBI PDB database for "MAGMG..."
@> Blast search completed in 4.5s.
@> Blast searching NCBI PDB database for "MSSKQ..."
@> Blast search completed in 4.8s.
@> Blast searching NCBI PDB database for "MSTRQ..."
@> Blast search completed in 5.1s.
@> Blast searching NCBI PDB database for "MEQSI..."
@> Blast search completed in 6.4s.
@> Blast searching NCBI PDB database for "

## Step 3: sequence alignment by MAFFT
- input file: sequences.fasta
- output file: seq_alignment.fasta

In [66]:
from Bio.Align.Applications import MafftCommandline as mftc
from Bio import AlignIO

#specify the executable location for mafft software
mafft_exe = "/usr/local/bin/mafft"
#directory of your input file
in_file = "/Users/Yoana/Dropbox_SBGrid/python/graduate_project/sequences.fasta"
mafft_cline = mftc(mafft_exe, input=in_file)

#mafft writes the alignment in stdout
stdout, stderr = mafft_cline()

#save data from stdout in seq_alignment.fasta
with open("seq_alignment.fasta", "w") as handle:
    handle.write(stdout)

align = AlignIO.read("seq_alignment.fasta", "fasta")

## Step 4: creating a color coded sequence alignment file
### Two options to visualize your sequence alignment:
- Jalview: http://www.jalview.org/download 
- ESPript: http://espript.ibcp.fr/ESPript/ESPript/index.php

#### Running ESPript
input file: espript.inp 

go_espript.csh containing:
#the executable location of the espript software (must be a Linux maxine):
- /Users/.../software/espript/ESPript_linux_x86-64 < espript.inp

#espript writes a ps file and converts to final pdf output file:
- ps2pdf espript.ps cse4_final_alignment.pdf

#Open a terminal and run: 
- source go_espript.csh


#### Helpful resources for the used modules:
- Biopython - (MAFFT)
http://biopython.org/DIST/docs/api/Bio.Align.Applications._Mafft.MafftCommandline-class.html
- Biopython - (Entrez)
http://biopython.org/DIST/docs/tutorial/Tutorial.html
https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch
- Biopython - (SeqIO)
http://biopython.org/wiki/SeqIO
- ProDy - Protein Dynamics & sequence analysis
http://prody.csb.pitt.edu/tutorials/structure_analysis/blastpdb.html
- ESPript 
http://espript.ibcp.fr/ESPript/ESPript/esp_userguide.php
- NCBI yourtube tutorials: 
https://www.youtube.com/results?search_query=ncbi